# Neomaril Training

This notebook give a exemple on how to use Neomaril to training a ML model

### NeomarilTrainingClient

It's where you can manage your trainining experiments

In [1]:
# Import the client
from neomaril_codex.training import NeomarilTrainingClient

In [2]:
# Start the client. We are reading the credentials in the NEOMARIL_TOKEN env variable

client = NeomarilTrainingClient()
client

2023-02-17 16:16:28.736 | INFO     | neomaril_codex._base:__init__:77 - Loading .env
2023-02-17 16:16:29.194 | INFO     | neomaril_codex._base:__init__:89 - Successfully connected to Neomaril


NeomarilTrainingClient(url="http://localhost:7070/api", version="1.0")

## NeomarilTrainingExperiment

It's where you can create a training experiment to find the best model

#### Custom training

With Custom training you have to create the training function.

In [3]:
# Creating a new training experiment
training = client.create_training_experiment('Teste notebook Training custom', # Experiment name, this is how you find your model in MLFLow
                                            'Classification', # Model type. Can be Classification, Regression or Unsupervised
                                            'Custom', # Training type. Can be Custom or AutoML
                                            group='datarisk' # This is the default group. Create a new one when using for a new project
                                            )

2023-02-17 16:16:30.364 | INFO     | neomaril_codex.training:create_training_experiment:466 - New Training inserted with hash Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3.


In [4]:
training

NeomarilTrainingExperiment(name="Teste notebook Training custom", 
                                                        group="datarisk", 
                                                        training_id="Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3",
                                                        training_type="Custom",
                                                        model_type=Classification
                                                        )

In [5]:
# With the experiment class we can create multiple model runs
PATH = './samples/train/'

run = training.run_training('First test', # Run name
                            PATH+'dados.csv', # Path to the file with training data
                            source_file=PATH+'app.py', # Path of the source file
                            requirements_file=PATH+'requirements.txt', # Path of the requirements file, 
#                           env=PATH+'.env'  #  File for env variables (this will be encrypted in the server)
#                           extra_files=[PATH+'utils.py'], # List with extra files paths that should be uploaded along (they will be all in the same folder)
                            training_reference='train_model',
                            python_version='3.9', # Can be 3.7 to 3.10
                            wait_complete=True
)

2023-02-17 16:16:32.503 | INFO     | neomaril_codex.training:__upload_training:283 - {"Message":"Training files have been uploaded. Execution id is '1'"}
2023-02-17 16:16:33.520 | INFO     | neomaril_codex.training:__execute_training:302 - Model training starting - Hash: Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3


Wating the training run.........

In [6]:
run.execution_data

{'ModelHash': 'Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3',
 'ExperimentName': 'Teste notebook Training custom',
 'GroupName': 'datarisk',
 'ModelType': 'Classification',
 'TrainingType': 'Custom',
 'ExecutionId': 1,
 'ExecutionState': 'Running',
 'InputPayload': '{\n    "trainingReference": "train_model",\n    "pythonVersion": "Python39",\n    "executionType": "neomaril-custom-training",\n    "experimentName": "Teste notebook Training custom",\n    "runName": "First test",\n    "basePath": "/app/store/datarisk/Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3/1",\n    "neomarilExecutionId": "1"\n}',
 'OutputPayload': '{}',
 'RunAt': '2023-02-17T19:16:31.9621030Z',
 'Status': 'Succeeded'}

In [7]:
run.get_status()

{'trainingExecutionId': '1',
 'Status': 'Succeeded',
 'Message': '{\n    "artifacts": "wasbs://mlflow-staging@datariskmlops.blob.core.windows.net/artifacts/1/1931b99907da4772b0a07f59ee3e727b/artifacts",\n    "mlflow_run_id": "1931b99907da4772b0a07f59ee3e727b"\n}'}

In [8]:
# When the run is finished you can download the model file
run.download_result()

# or promote promete it to a deployed model

PATH = './samples/syncModel/'

model = run.promote_model('Teste notebook promoted custom', # model_name
                            'score', # name of the scoring function
                            PATH+'app.py', # Path of the source file
                            PATH+'schema.json', # Path of the schema file, but it could be a dict
#                           env=PATH+'.env'  #  File for env variables (this will be encrypted in the server)
#                           extra_files=[PATH+'utils.py'], # List with extra files paths that should be uploaded along (they will be all in the same folder)
                            operation="Sync" # Can be Sync or Async
)

2023-02-17 16:20:46.431 | INFO     | neomaril_codex._base:download_result:257 - Output saved in ./output_1.zip
2023-02-17 16:20:48.932 | INFO     | neomaril_codex.training:__upload_model:112 - Model 'Teste notebook promoted custom' promoted from Teb54da9c0ba455487728929e32b5fdd24ebd43fa1024a90aa98c5bf2a3d1bb3 - Hash: "M85f50255c64401ea7aa39bb87fd284aba8acd91f9f54649932516706d54eb95"
2023-02-17 16:20:49.436 | INFO     | neomaril_codex.training:__host_model:134 - Model host in process - Hash: M85f50255c64401ea7aa39bb87fd284aba8acd91f9f54649932516706d54eb95
2023-02-17 16:20:49.437 | INFO     | neomaril_codex.model:__init__:31 - Loading .env


In [9]:
model

NeomarilModel(name="Teste notebook promoted custom", group="datarisk", 
                                status="Building",
                                model_id="M85f50255c64401ea7aa39bb87fd284aba8acd91f9f54649932516706d54eb95",
                                operation="Sync",
                                schema={
  "mean_radius": 17.99,
  "mean_texture": 10.38,
  "mean_perimeter": 122.8,
  "mean_area": 1001.0,
  "mean_smoothness": 0.1184,
  "mean_compactness": 0.2776,
  "mean_concavity": 0.3001,
  "mean_concave_points": 0.1471,
  "mean_symmetry": 0.2419,
  "mean_fractal_dimension": 0.07871,
  "radius_error": 1.095,
  "texture_error": 0.9053,
  "perimeter_error": 8.589,
  "area_error": 153.4,
  "smoothness_error": 0.006399,
  "compactness_error": 0.04904,
  "concavity_error": 0.05373,
  "concave_points_error": 0.01587,
  "symmetry_error": 0.03003,
  "fractal_dimension_error": 0.006193,
  "worst_radius": 25.38,
  "worst_texture": 17.33,
  "worst_perimeter": 184.6,
  "worst_area":

#### AutoML

With AutoML you just need to upload the data and some configuration

In [10]:
# Creating a new training experiment
training = client.create_training_experiment('Teste notebook Training AutoML', # Experiment name
                                            'Classification', # Model type. Can be Classification, Regression or Unsupervised
                                            'AutoML', # Training type. Can be Custom or AutoML
                                            group='datarisk' # This is the default group. Create a new one when using for a new project
                                            )

PATH = './samples/autoML/'

run = training.run_training('First test', # Run name
                            PATH+'dados.csv', # Path to the file with training data
                            conf_dict=PATH+'conf.json', # Path of the configuration file
                            wait_complete=True
)

2023-02-17 16:20:51.232 | INFO     | neomaril_codex.training:create_training_experiment:466 - New Training inserted with hash T3b4eba46d564cdc91d9390c62240484328cde0e6f134affbc80d21ca48e5a1b.
2023-02-17 16:20:52.890 | INFO     | neomaril_codex.training:__upload_training:283 - {"Message":"Training files have been uploaded. Execution id is '2'"}
2023-02-17 16:20:53.863 | INFO     | neomaril_codex.training:__execute_training:302 - Model training starting - Hash: T3b4eba46d564cdc91d9390c62240484328cde0e6f134affbc80d21ca48e5a1b


Wating the training run...........

2023-02-17 16:26:08.344 | ERROR    | neomaril_codex._base:get_status:221 - "Given an invalid JSON: Additional text encountered after finished reading JSON content: [. Path '', line 1, position 183."


ExecutionError: Execution "2" unavailable

In [ ]:
run

NeomarilTrainingExecution(exec_id="2", status="Succeeded")

In [ ]:
run.get_status()

{'trainingExecutionId': '2',
 'Status': 'Succeeded',
 'Message': '{\n    "artifacts": "wasbs://mlflow-dev@datariskmlops.blob.core.windows.net/artifacts/2/f7ca166bbf8f48c19ec4c296d6f41b08/artifacts",\n    "mlflow_run_id": "f7ca166bbf8f48c19ec4c296d6f41b08"\n}'}

In [ ]:
run

NeomarilTrainingExecution(exec_id="2", status="Succeeded")

In [ ]:
# Promote a AutoML model is a lot easier

PATH = './samples/syncModel/'

model = run.promote_model('Teste notebook promoted autoML', # model_name
                            operation="Async" # Can be Sync or Async
)

2023-02-17 13:57:47.789 | INFO     | neomaril_codex.training:__upload_model:112 - Model 'Teste notebook promoted autoML' promoted from T1a6d75c54dd40188275edea951bb04508740293bd2f402da55f1e08af318a57 - Hash: "Mf859a32ac3547a58c6f93f760904a95b5136add1846455ab5dba93009280ce1"
2023-02-17 13:57:47.819 | INFO     | neomaril_codex.training:__host_model:134 - Model host in process - Hash: Mf859a32ac3547a58c6f93f760904a95b5136add1846455ab5dba93009280ce1
2023-02-17 13:57:47.821 | INFO     | neomaril_codex.model:__init__:31 - Loading .env


In [ ]:
model

NeomarilModel(name="Teste notebook promoted autoML", group="datarisk", 
                                status="Building",
                                model_id="Mf859a32ac3547a58c6f93f760904a95b5136add1846455ab5dba93009280ce1",
                                operation="Async",
                                schema={}
                                )